In [ ]:
import os
import tqdm
import metpy.calc
from metpy.units import units
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import cartopy.io.shapereader as shpreader

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from utils import read_config
from plotting import ptype_hist
from qc import wetbulb_filter

In [ ]:
config = 'config/mPING.yml'
# config = 'config/mPING_hourafter.yml'

conf = read_config(config)
data = pd.read_parquet(conf['data_path'])
dataset = conf['dataset']
image_path = conf['image_path']

## Add Lat-Lon to data

In [ ]:
def add_latlon(df, date="20190202", hour="14", path_rap="/glade/p/cisl/aiml/conv_risk_intel/rap_ncei_height"):
    '''
    Add latitude and longitude columns to the mPING data.
    '''
    x = df['x_m'].values      #get x and y indices from data
    y = df['y_m'].values
    total_pts = df.shape[0]
    
    rap_data = xr.open_dataset(os.path.join(path_rap, date, f"rap_130_{date}_{hour}00_000.nc")) #open example RAP dataset
    grid_lons = np.subtract(rap_data.longitude.values, 360)
    grid_lats = rap_data.latitude.values
    
    lats = []
    lons = []
    for i in range(total_pts): #calculate lat and lon points based on the indices
        lats.append(grid_lats[y[i], x[i]])
        lons.append(grid_lons[y[i], x[i]])
    
    df['lon'], df['lat'] = np.array(lons), np.array(lats)
    
    return df

In [ ]:
data = add_latlon(data)

### Plot

In [ ]:
projection = ccrs.LambertConformal(central_longitude=-95,
                                  central_latitude=25)
res = '50m'  #'10m' is another option
fig, ax = plt.subplots(
    nrows=1, 
    ncols=1,
    figsize=(12,7),
    constrained_layout=True, 
    subplot_kw={'projection': projection})

lonW = -125
lonE = -63
latS = 20
latN = 50
ax.set_extent([lonW, lonE, latS, latN])
ax.add_feature(cfeature.LAND.with_scale(res))
ax.add_feature(cfeature.OCEAN.with_scale(res))
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.LAKES.with_scale(res), alpha=0.5)
ax.add_feature(cfeature.STATES.with_scale(res))

ax.scatter(data['lon'], 
           data['lat'], 
           c='k', 
           s=10,
           alpha=1,
           transform=ccrs.PlateCarree(),
           label=f"All data n={data['lon'].shape[0]}")

plt.legend(markerscale=2, fontsize=16, loc="lower right")
plt.title(f"{dataset}", fontsize=22)
plt.savefig(f'{image_path}all_data_simple_map.png', dpi=300, bbox_inches='tight')
plt.show()

## Continental US filter

Only keep points within the continental United States.

In [ ]:
def usa_mask(df):
    '''
    Function that flags points that are in the continental United States.
    '''
    df_lons, df_lats = df['lon'].values, df['lat'].values
    total_pts = df.shape[0]
    points = np.array([Point(lon, lat) for lon, lat in zip(df_lons, df_lats)]) #create list of shapely points
    
    shapefile = shpreader.natural_earth(resolution='110m',                
                                       category='cultural',
                                       name='admin_0_countries')   #open natural earth shapefile
    geo_df = gpd.read_file(shapefile)
    cont_usa = geo_df[geo_df['ADMIN'] == 'United States of America']['geometry'].values[0] #extract continental US polygon
    cont_usa = list(cont_usa.geoms)[0]
    
    mask = np.zeros(total_pts)
    
    for i in tqdm.tqdm(range(total_pts)):  #flip 0's to 1's if the point is within the US
        if cont_usa.contains(points[i]):
            mask[i] = 1
            
    df['usa'] = mask
        
    return df

In [ ]:
data = usa_mask(data)

### Plot

In [ ]:
projection = ccrs.LambertConformal(central_longitude=-95,
                                  central_latitude=25)
res = '50m'  #'10m' is another option
fig, ax = plt.subplots(
    nrows=1, 
    ncols=1,
    figsize=(12,7),
    constrained_layout=True, 
    subplot_kw={'projection': projection})

lonW = -125
lonE = -63
latS = 20
latN = 50
ax.set_extent([lonW, lonE, latS, latN])
ax.add_feature(cfeature.LAND.with_scale(res))
ax.add_feature(cfeature.OCEAN.with_scale(res))
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.LAKES.with_scale(res), alpha=0.5)
ax.add_feature(cfeature.STATES.with_scale(res))

ax.scatter(data['lon'][data['usa'] == 1], 
           data['lat'][data['usa'] == 1], 
           c='k', 
           s=10,
           alpha=1,
           transform=ccrs.PlateCarree(),
           label=f"All data n={data['lon'][data['usa'] == 1].shape[0]}")

plt.legend(markerscale=2, fontsize=16, loc="lower right")
plt.title(f"{dataset}", fontsize=22)
plt.savefig(f'{image_path}usa_pts_example.png', dpi=300, bbox_inches='tight')
plt.show()

## Precipitation Filter

Ideas 
- Filter if TOTAL_PRECIP > 0 (not all files have this variable)
- Filter if any CRAIN, CSNOW, CICEP, CFRZR = 1

In [ ]:
def filter_precip(df):
    '''
    Function that flags points where the RAP model
    indicates precipitation is occurring. Precipitation can
    be any ptype.
    '''
    total_pts = df.shape[0]
    mask = np.zeros(total_pts)
    
    crain = df.CRAIN.values  #get categorical precip values
    csnow = df.CSNOW.values
    cfrzr = df.CFRZR.values
    cicep = df.CICEP.values
    
    for i in tqdm.tqdm(range(total_pts)):
        if any([crain[i] == 1.0, csnow[i] == 1.0, cicep[i] == 1.0, cfrzr[i] == 1.0]):
                mask[i] = 1
    
    df['cprecip'] = mask
                
    return df

In [ ]:
data = filter_precip(data)

### Plot

In [ ]:
projection = ccrs.LambertConformal(central_longitude=-95,
                                  central_latitude=25)
res = '50m'  #'10m' is another option
fig, ax = plt.subplots(
    nrows=1, 
    ncols=1,
    figsize=(12,7),
    constrained_layout=True, 
    subplot_kw={'projection': projection})

lonW = -125
lonE = -63
latS = 20
latN = 50
ax.set_extent([lonW, lonE, latS, latN])
ax.add_feature(cfeature.LAND.with_scale(res))
ax.add_feature(cfeature.OCEAN.with_scale(res))
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.LAKES.with_scale(res), alpha=0.5)
ax.add_feature(cfeature.STATES.with_scale(res))

ax.scatter(data['lon'][(data['cprecip'] == 1) & (data['usa'] == 1)], 
           data['lat'][(data['cprecip'] == 1) & (data['usa'] == 1)], 
           c='k', 
           s=10,
           alpha=1,
           transform=ccrs.PlateCarree(),
           label=f"All data n={data['lon'][(data['cprecip'] == 1) & (data['usa'] == 1)].shape[0]}")

plt.legend(markerscale=2, fontsize=16, loc="lower right")
plt.title(f"{dataset}", fontsize=22)
plt.savefig(f'{image_path}usa_precip_example.png', dpi=300, bbox_inches='tight')
plt.show()

## Wet Bulb Temperature Filter

If there is a frozen pytpe present and the wet bulb temperature is greater than 5 C, remove that pytpe

__Notes:__
- Dewpoint depression 2m variable is inaccurate in the mPING data (seems like dewpoint temp was in K and temp was in C because of the large negative values), this can be fixed by adding 273.15 to the value but I found that doing the original subtraction is more accurate

In [ ]:
config = 'config/mPING.yml'
# config = 'config/mPING_hourafter.yml'

conf = read_config(config)
data = pd.read_parquet(conf['qc2_data_path'])
dataset = conf['dataset']
image_path = conf['image_path']

In [ ]:
thresholds = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
for threshold in thresholds:    
    data = wetbulb_filter(data, threshold=threshold)

In [ ]:
cols = list(data.columns)
for col in cols:
    print(col)

In [ ]:
data.to_parquet(f"/glade/scratch/jwillson/winter-ptype/{dataset}_interpolated_QC2.parquet")

## Test if the QC improves data quality

In [ ]:
config = 'config/mPING.yml'
# config = 'config/mPING_hourafter.yml'

conf = read_config(config)
data = pd.read_parquet(conf['qc2_data_path'])
dataset = conf['dataset']
image_path = conf['image_path']

In [ ]:
print(data.shape)
# usa_df = data[data['usa'] == 1.0]
qc1_df = data[(data['cprecip'] == 1.0) & (data['usa'] == 1.0)]
qc2_df = data[(data['wetbulb5.0_filter'] == 0.0) & (data['usa'] == 1.0)]
print(qc1_df.shape)
print(qc2_df.shape)

### P-type surface temperature distributions

In [ ]:
col = 'TEMP_C_0_m'    #, 'T_DEWPOINT_C_0_m', 'UGRD_m/s_0_m', 'VGRD_m/s_0_m']
name = '0m_temp'    #, '0m_temp_dew', '0m_ugrd', '0m_vgrd']
bins = np.arange(-40,40,1)
dfs = [qc1_df, qc2_df]
save_paths = [f"{image_path}{name}_QC1hist.png", f"{image_path}{name}_QC2hist.png"]
classes = ['Rain', 'Snow', 'Ice Pellets', 'Freezing Rain']
qc_names = ['QC1', 'QC2']

for i, df in enumerate(dfs):
    qc_ra = df[col][df['ra_percent'] > 0.0]
    qc_sn = df[col][df['sn_percent'] > 0.0]
    qc_pl = df[col][df['pl_percent'] > 0.0]
    qc_fzra = df[col][df['fzra_percent'] > 0.0]
    qc_ptypes = [qc_ra, qc_sn, qc_pl, qc_fzra]
    
    ra = data[col][data['ra_percent'] > 0.0]
    sn = data[col][data['sn_percent'] > 0.0]
    pl = data[col][data['pl_percent'] > 0.0]
    fzra = data[col][data['fzra_percent'] > 0.0]
    ptypes = [ra, sn, pl, fzra]
    
    fig, ax = plt.subplots(2, 2, figsize=(12,10), tight_layout=True)
    
    for j in range(len(ptypes)):
        ax.ravel()[j].hist(ptypes[j], bins=bins, label=f"{dataset} Original")
        ax.ravel()[j].hist(qc_ptypes[j], bins=bins, color='k', label=f"{dataset} {qc_names[i]}")
        ax.ravel()[j].set_yscale('log')
        if j > 1:
            ax.ravel()[j].set_xlabel("0 m Temperature ($\degree$C)", fontsize=16)
        ax.ravel()[j].tick_params(axis='x', labelsize=14)
        ax.ravel()[j].tick_params(axis='y', labelsize=14)
        ax.ravel()[j].set_title(f"{classes[j]}", fontsize=16)
        if j == 0:
            ax.ravel()[j].legend(fontsize=12)
    
    plt.savefig(save_paths[i], dpi=300, bbox_inches="tight")
    plt.show()

### Wet Bulb Threshold Study

In [ ]:
thresholds = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
ra_le10_vals = []
sn_ge10_vals = []
pl_ge10_vals = []
fzra_ge10_vals = []
for threshold in thresholds: 
    test_df = data[(data[f"wetbulb{threshold}_filter"] == 0.0) & (data['usa'] == 1.0)]
    testdf_total = test_df.shape[0]
    testdf_ra5 = test_df[(test_df['ra_percent'] > 0.0) & (test_df['TEMP_C_0_m'] <= -10.0)].shape[0]
    testdf_sn10 = test_df[(test_df['sn_percent'] > 0.0) & (test_df['TEMP_C_0_m'] >= 10.0)].shape[0]
    testdf_pl10 = test_df[(test_df['pl_percent'] > 0.0) & (test_df['TEMP_C_0_m'] >= 10.0)].shape[0]
    testdf_fzra10 = test_df[(test_df['fzra_percent'] > 0.0) & (test_df['TEMP_C_0_m'] >= 10.0)].shape[0]

    ra_le10_vals.append(testdf_ra5/testdf_total*100.0)
    sn_ge10_vals.append(testdf_sn10/testdf_total*100.0)
    pl_ge10_vals.append(testdf_pl10/testdf_total*100.0)
    fzra_ge10_vals.append(testdf_fzra10/testdf_total*100.0)

In [ ]:
fig, ax = plt.subplots(figsize=(12,7), tight_layout=True)
ax.plot(thresholds, ra_le10_vals, '.-r', markersize=10, label='Percent Rain Obs <= 10 C')
ax.plot(thresholds, sn_ge10_vals, '.-c', markersize=10, label='Percent Snow Obs >= 10 C')
ax.plot(thresholds, pl_ge10_vals, '.-b', markersize=10, label='Percent Ice Pellets Obs >= 10 C')
ax.plot(thresholds, fzra_ge10_vals, '.-k', markersize=10, label='Percent Freezing Rain Obs >= 10 C')
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.set_ylabel('Percentage', fontsize=16)
ax.set_xlabel('Wet Bulb Temp Threshold ($\degree$C)', fontsize=16)
ax.set_title(f"{dataset}", fontsize=22)
ax.legend(fontsize=12)
plt.savefig(f"{image_path}thresh_percent10.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
thresholds = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
colors = ['y', 'g', 'c', 'orange', 'b', 'r', 'm', 'k']
classes = ['Rain', 'Snow', 'Ice Pellets', 'Freezing Rain']
temp_thresh = 10.0
col = 'TEMP_C_0_m'
fig, ax = plt.subplots(2, 2, figsize=(12,10), tight_layout=True)
bins = np.arange(-40,40,1)

for i, threshold in enumerate(thresholds[::-1]): 
    test_df = data[(data[f"wetbulb{threshold}_filter"] == 0.0) & (data['usa'] == 1.0)]
    testdf_ra10 = test_df[col][(test_df['ra_percent'] > 0.0)]
    testdf_sn10 = test_df[col][(test_df['sn_percent'] > 0.0)]
    testdf_pl10 = test_df[col][(test_df['pl_percent'] > 0.0)]
    testdf_fzra10 = test_df[col][(test_df['fzra_percent'] > 0.0)]
    
    dfs = [testdf_ra10, testdf_sn10, testdf_pl10, testdf_fzra10]
    
    for j, df in enumerate(dfs):
        ax.ravel()[j].hist(df, bins=bins, color=colors[i], label=str(threshold))
        ax.ravel()[j].set_yscale('log')
        if j > 1:
            ax.ravel()[j].set_xlabel("0 m Temperature ($\degree$C)", fontsize=16)
        ax.ravel()[j].tick_params(axis='x', labelsize=14)
        ax.ravel()[j].tick_params(axis='y', labelsize=14)
        ax.ravel()[j].set_title(f"{classes[j]}", fontsize=16)
        if j == 0:
            ax.ravel()[j].legend(fontsize=12)
    
# plt.savefig(f"{image_path}comb_thresh_hist10.png", dpi=300, bbox_inches="tight")
plt.show()

### Numerical results

In [ ]:
testdf_total = test_df.shape[0]
testdf_ra10 = test_df[(test_df['ra_percent'] > 0.0) & (test_df['TEMP_C_0_m'] <= -10.0)].shape[0]
testdf_sn10 = test_df[(test_df['sn_percent'] > 0.0) & (test_df['TEMP_C_0_m'] >= 10.0)].shape[0]
testdf_pl10 = test_df[(test_df['pl_percent'] > 0.0) & (test_df['TEMP_C_0_m'] >= 10.0)].shape[0]
testdf_fzra10 = test_df[(test_df['fzra_percent'] > 0.0) & (test_df['TEMP_C_0_m'] >= 10.0)].shape[0]

test_percent_ra = testdf_ra10/testdf_total*100.0
test_percent_sn = testdf_sn10/testdf_total*100.0
test_percent_pl = testdf_pl10/testdf_total*100.0
test_percent_fzra = testdf_fzra10/testdf_total*100.0

print(round(test_percent_ra, 5))
print(round(test_percent_sn, 5))
print(round(test_percent_pl, 5))
print(round(test_percent_fzra, 5))

In [ ]:
data_total = data.shape[0]
data_ra10 = data[(data['ra_percent'] > 0.0) & (data['TEMP_C_0_m'] <= -10.0)].shape[0]
data_sn10 = data[(data['sn_percent'] > 0.0) & (data['TEMP_C_0_m'] >= 10.0)].shape[0]
data_pl10 = data[(data['pl_percent'] > 0.0) & (data['TEMP_C_0_m'] >= 10.0)].shape[0]
data_fzra10 = data[(data['fzra_percent'] > 0.0) & (data['TEMP_C_0_m'] >= 10.0)].shape[0]

data_percent_ra = data_ra10/data_total*100.0
data_percent_sn = data_sn10/data_total*100.0
data_percent_pl = data_pl10/data_total*100.0
data_percent_fzra = data_fzra10/data_total*100.0

print(round(data_percent_ra, 5))
print(round(data_percent_sn, 5))
print(round(data_percent_pl, 5))
print(round(data_percent_fzra, 5))

In [ ]:
total_diff = (testdf_total-data_total)/data_total*100.0
ra10_diff = (test_percent_ra-data_percent_ra)/data_percent_ra*100.0
sn10_diff = (test_percent_sn-data_percent_sn)/data_percent_sn*100.0
pl10_diff = (test_percent_pl-data_percent_pl)/data_percent_pl*100.0
fzra10_diff = (test_percent_fzra-data_percent_fzra)/data_percent_fzra*100.0

print(round(total_diff, 2))
print(round(ra10_diff, 2))
print(round(sn10_diff, 2))
print(round(pl10_diff, 2))
print(round(fzra10_diff, 2))